# backtrader - 지정가 주문 다루기

backtrader를 사용해서 지정가 주문을 다루는 예제코드입니다.
아래 내용들을 다루고 있습니다.
- 지정가 주문 제출
- 조건부 지정가 주문 제출
- 미체결 주문 조회
- 정정 주문 및 취소 주문 제출 

### 주문 유형

backtrader에는 다음의 주문 유형이 정의되어 있습니다.
- `Order.Market`: 시장가 주문 (기본값)
- `Order.Limit`: 지정가 주문
- `Order.Stop`: 지정된 가격에 도달했을 때 시장가 주문 제출
- `Order.StopLimit`: 지정된 가격에 도달했을 때 `pricelimit`으로 지정된 가격으로 지정가 주문 제출


`valid` 파라미터와 조합하여 주문의 유효기간을 설정할 수 있습니다.
- `None`: 취소할 때까지 주문이 계속 살아있음. 국내 시장의 경우 지원하지 않는 유형. (기본값)
- `datetime.datetime` or `datetime.date`: 지정할 날짜까지 주문이 계속 살아있음. 마찬가지로 국내 시장의 경우 거래일이 넘어갈 때까지 남아있는 주문 유형은 없음
- `Order.DAY` or `0` or timedelta(): 당일 장 마감시간 까지 유효
- 숫자값: matplotlib coding에서 x축의 해당하는 시간값을 의미. 


국내 시장에서 제공되는 주문유형인 조건부 지정가(장 마감시까지 체결이 안된 경우 시장가로 체결)는 제공되지 않으므로 주의해야 합니다. 또한 지정가 주문을 사용하는 경우 valid 파라미터는 항상 `Order.DAY`로 사용해야 합니다.

### 지정가 주문 제출

지정가 주문을 체결될 수 없는 가격 제출 (현재 타임라인의 저가보다 10틱아래)에 제출하여 미체결 상태로 장 종료 시점 까지 남아있는 경우를 만들었습니다.
- buy(price=가격, exectype=지정가, valid=오늘한정, size=매수수량)
- get_order_open() - 미체결 주문 조회
- getposition(data) - 보유포지션 조회

In [62]:
import backtrader as bt
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data
from pyqqq.utils.compute import get_krx_tick_size

class St(bt.Strategy):
    last_trade_id = 1

    params = {
        'short_period': 30,
        'long_period': 120,
    }

    def __init__(self):
        # 지표는 __init__에서 선언해야 함
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)
        self.last_trade_id = 1

    def log(self, s):
        # 현재 시간은 self.data.datetime 으로 접근가능
        dt = self.data.datetime.datetime()
        print(f"{dt.isoformat()}: {s}")

    def next(self):
        # self.log(f"open={self.data.open[0]} high={self.data.high[0]} low={self.data.low[0]} close={self.data.close[0]} volume={self.data.volume[0]}")

        position = self.broker.getposition(data=self.data)
        if position:
            self.log(f"Position size={position.size} price={position.price} value={position.size*position.price}")

        pending_orders = self.broker.get_orders_open()
        for order in pending_orders:
            self.log(f"[Pending Order] {order.data._name} size={order.size} price={order.price} tradeid={order.tradeid}")

        if self.short_ma > self.long_ma and len(pending_orders) == 0:
            tick = get_krx_tick_size(self.data.close[0], etf_etn=False)
            target_price = self.data.low[0] - tick*10

            self.buy(price=target_price, exectype=bt.Order.Limit, valid=bt.Order.DAY, size=1, tradeid=self.last_trade_id)
            self.last_trade_id += 1
            self.log(f"Send buy order at {target_price}")

        elif self.short_ma < self.long_ma and position.size > 0:
            self.sell()

    def notify_order(self, order):
        exectypes = ['Market', 'Close', 'Limit', 'Stop', 'StopLimit', 'StopTrail', 'StopTrailLimit', 'Historical']
        statuses =  ['Created', 'Submitted', 'Accepted', 'Partial', 'Completed', 'Canceled', 'Expired', 'Margin', 'Rejected']

        self.log(f"[{'Buy' if order.isbuy() else 'Sell'} Order] {order.data._name} size={order.size} exectype={exectypes[order.exectype]} price={order.price} status={statuses[order.status]} tradeid={order.tradeid} current_price={self.data.low[0]}")


asset_code = '005930'
target_date = dtm.datetime.today() - dtm.timedelta(days=1)
dfs = get_all_day_data(target_date.date(), [asset_code])
df = dfs[asset_code]

cerebro = bt.Cerebro()

cerebro.adddata(bt.feeds.PandasData(dataname=df), name=asset_code)
initial_cash = 10_000_000
cerebro.broker.setcash(initial_cash)
cerebro.addstrategy(St)

cerebro.run()

print(f"Initial Portfolio Value: {initial_cash}")
print(f"Final Portfolio Value: {cerebro.broker.getvalue()}")

2024-05-22T10:59:00: Send buy order at 77400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Submitted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Accepted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:01:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:02:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:03:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:04:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:05:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:06:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:07:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:08:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:09:00: [Pending

### 지정가 - 취소주문, 정정주문

backtrader는 정정주문을 지원하지 않습니다. 정정주문을 하려면, 기존 주문을 취소하고 새로운 주문을 내려야 합니다.
- self.broker.getposition(data): 보유포지션 조회
- self.get_orders_open(): 미체결 주문 조회
- self.cancel(order): 미체결 주문 쥐소

In [69]:
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    last_trade_id = 1

    params = {
        'short_period': 30,
        'long_period': 120,
    }

    def __init__(self):
        # 지표는 __init__에서 선언해야 함
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)
        self.last_trade_id = 1

    def log(self, s):
        # 현재 시간은 self.data.datetime 으로 접근가능
        dt = self.data.datetime.datetime()
        print(f"{dt.isoformat()}: {s}")

    def next(self):
        position = self.broker.getposition(data=self.data)
        if position:
            self.log(f"Position size={position.size} price={position.price} value={position.size*position.price}")

        pending_orders = self.broker.get_orders_open()
        for order in pending_orders:
            self.log(f"[Pending Order] {order.data._name} size={order.size} price={order.price} tradeid={order.tradeid}")

        # Golden cross 상황에 포지션 없으면 매수 주문
        if self.short_ma > self.long_ma and not position:
            if len(pending_orders) == 0:
                # 주문이 없으면 체결될 수 없는 낮은 가격으로 매수 주문
                tick = get_krx_tick_size(self.data.close[0], etf_etn=False)
                target_price = self.data.low[0] - tick*10

                self.buy(price=target_price, exectype=bt.Order.Limit, valid=bt.Order.DAY, size=1, tradeid=self.last_trade_id)
                self.last_trade_id += 1
                self.log(f"Send buy order at {target_price}")
            else:
                # 기존 주문을 취소하고 체결 될 수 있는 고가로 새로 주문 제출
                order = pending_orders[0]
                self.cancel(order)

                self.buy(price=self.data.low[0], exectype=bt.Order.Limit, valid=bt.Order.DAY, size=1, tradeid=self.last_trade_id)
                self.last_trade_id += 1
                self.log(f"Cancel previous order and send new buy order at {self.data.low[0]}")



        elif self.short_ma < self.long_ma and position.size > 0:
            self.sell()


    def notify_order(self, order):
        exectypes = ['Market', 'Close', 'Limit', 'Stop', 'StopLimit', 'StopTrail', 'StopTrailLimit', 'Historical']
        statuses =  ['Created', 'Submitted', 'Accepted', 'Partial', 'Completed', 'Canceled', 'Expired', 'Margin', 'Rejected']

        self.log(f"[{'Buy' if order.isbuy() else 'Sell'} Order] {order.data._name} size={order.size} exectype={exectypes[order.exectype]} price={order.price} status={statuses[order.status]} tradeid={order.tradeid} current_price={self.data.low[0]}")


asset_code = '005930'
target_date = dtm.datetime.today() - dtm.timedelta(days=1)
dfs = get_all_day_data(target_date.date(), [asset_code])
df = dfs[asset_code]

cerebro = bt.Cerebro()

cerebro.adddata(bt.feeds.PandasData(dataname=df), name=asset_code)
initial_cash = 10_000_000
cerebro.broker.setcash(initial_cash)
cerebro.addstrategy(St)

cerebro.run()

print(f"Initial Portfolio Value: {initial_cash}")
print(f"Final Portfolio Value: {cerebro.broker.getvalue()}")

2024-05-22T10:59:00: Send buy order at 77400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Submitted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Accepted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:00:00: Cancel previous order and send new buy order at 78400.0
2024-05-22T11:01:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Canceled tradeid=1 current_price=78400.0
2024-05-22T11:01:00: [Buy Order] 005930 size=1 exectype=Limit price=78400.0 status=Submitted tradeid=2 current_price=78400.0
2024-05-22T11:01:00: [Buy Order] 005930 size=1 exectype=Limit price=78400.0 status=Accepted tradeid=2 current_price=78400.0
2024-05-22T11:01:00: [Buy Order] 005930 size=1 exectype=Limit price=78400.0 status=Completed tradeid=2 current_price=78400.0
2024-05-22T11:01:00: Position size=1 price=78400.0 val

### 조건부 지정가

조건부 지정가 방식도 backtrader에 존재하지 않습니다. 
지정가 주문에 대한 미체결주문이 동시호가시간에 남아 있는 경우 취소하고 종가로 제출하여 같은 효과를 구현할 수 있습니다.

In [70]:
import datetime as dtm
from pyqqq.data.minutes import get_all_day_data

class St(bt.Strategy):
    last_trade_id = 1

    params = {
        'short_period': 30,
        'long_period': 120,
    }

    def __init__(self):
        # 지표는 __init__에서 선언해야 함
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)
        self.last_trade_id = 1

    def log(self, s):
        # 현재 시간은 self.data.datetime 으로 접근가능
        dt = self.data.datetime.datetime()
        print(f"{dt.isoformat()}: {s}")

    def next(self):
        position = self.broker.getposition(data=self.data)
        if position:
            self.log(f"Position size={position.size} price={position.price} value={position.size*position.price}")

        pending_orders = self.broker.get_orders_open()
        for order in pending_orders:
            self.log(f"[Pending Order] {order.data._name} size={order.size} price={order.price} tradeid={order.tradeid}")

        # Golden cross 상황에 포지션 없으면 매수 주문
        if self.short_ma > self.long_ma and not position:
            if len(pending_orders) == 0:
                # 주문이 없으면 체결될 수 없는 낮은 가격으로 매수 주문
                tick = get_krx_tick_size(self.data.close[0], etf_etn=False)
                target_price = self.data.low[0] - tick*10

                self.buy(price=target_price, exectype=bt.Order.Limit, valid=bt.Order.DAY, size=1, tradeid=self.last_trade_id)
                self.last_trade_id += 1
                self.log(f"Send buy order at {target_price}")



        current_time = self.data.datetime.time()

        if len(pending_orders) > 0 and current_time.hour == 15 and current_time.minute == 19:
            for order in pending_orders:
                self.cancel(order)
                self.log(f"Cancel and make new market order {order.tradeid}")
                self.buy(tradeid=self.last_trade_id, size=1, exectype=bt.Order.Close)
                self.last_trade_id += 1

    def notify_order(self, order):
        exectypes = ['Market', 'Close', 'Limit', 'Stop', 'StopLimit', 'StopTrail', 'StopTrailLimit', 'Historical']
        statuses =  ['Created', 'Submitted', 'Accepted', 'Partial', 'Completed', 'Canceled', 'Expired', 'Margin', 'Rejected']

        self.log(f"[{'Buy' if order.isbuy() else 'Sell'} Order] {order.data._name} size={order.size} exectype={exectypes[order.exectype]} price={order.price} status={statuses[order.status]} tradeid={order.tradeid} current_price={self.data.low[0]}")


asset_code = '005930'
target_date = dtm.datetime.today() - dtm.timedelta(days=1)
dfs = get_all_day_data(target_date.date(), [asset_code])
df = dfs[asset_code]

cerebro = bt.Cerebro()

cerebro.adddata(bt.feeds.PandasData(dataname=df), name=asset_code)
initial_cash = 10_000_000
cerebro.broker.setcash(initial_cash)
cerebro.addstrategy(St)

cerebro.run()

print(f"Initial Portfolio Value: {initial_cash}")
print(f"Final Portfolio Value: {cerebro.broker.getvalue()}")

2024-05-22T10:59:00: Send buy order at 77400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Submitted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Buy Order] 005930 size=1 exectype=Limit price=77400.0 status=Accepted tradeid=1 current_price=78400.0
2024-05-22T11:00:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:01:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:02:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:03:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:04:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:05:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:06:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:07:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:08:00: [Pending Order] 005930 size=1 price=77400.0 tradeid=1
2024-05-22T11:09:00: [Pending